In [5]:
import pandas as pd
import numpy as np
from pymoo.algorithms.nsga3 import NSGA3
from pymoo.algorithms.unsga3 import UNSGA3
from pymoo.optimize import minimize
import pickle

from pymoo.util.termination.f_tol import MultiObjectiveSpaceToleranceTermination

from notebooks.optimization_problems.constraints import Requirements

In [6]:
columns = ['strand_name', 'tof',
           'r_a_x', 'r_a_y', 'r_a_z',
           'v_a_x', 'v_a_y', 'v_a_z',
           'r_b_x', 'r_b_y', 'r_b_z',
           'd',
           'r_ab_sff_x', 'r_ab_sff_y', 'r_ab_sff_z']

file_path = "raw/eo_telesat_post_processed.h5"

row_limit = -1

store = pd.HDFStore(file_path)

instances = store.select('contact_instances', 'columns = %s' % str(columns), stop = row_limit)

# Sort by pass id
instances = instances.sort_index(0)

In [7]:
case, target = pickle.load(open('raw/eo_settings.pkl', 'rb'))

import util
util.print_targets(case)

Targets per orbit
T_target: 960.00 Gb (min: 624.00 Gb, max: 1296.00 Gb)
L_target: 0.00 s
E_target: 41.16 kJ (max: -0.00 kJ)
P_target: 0.00 s


In [8]:
from notebooks.optimization_problems.design_vector import design_vector_default_scm, SystemParameters

sys_param = SystemParameters()
sys_param.fc_Hz = target['frequency']
sys_param.GT_dBK = target['GT_dBK']
sys_param.margin_dB = 3.0
sys_param.B_Hz_array = np.array([0.1, 0.5, 1, 10, 50, 100, 200, 300]) * 1e6
sys_param.Gtx_dBi_bounds = (3., 43.)
sys_param.Ptx_dBm_bounds = (20., 43.)

modcods_df = pd.read_pickle('../dvbs2.pkl')

sys_param.EsN0_req_dB_array = modcods_df[['isend', 'isend_200MHz', 'isend_300MHz']].to_numpy()
sys_param.eta_bitsym_array = modcods_df[['eta', 'eta_200MHz', 'eta_300MHz']].to_numpy()
sys_param.eta_maee_array = modcods_df[['maee_12', 'maee_12_200MHz', 'maee_12_300MHz']].to_numpy()

requirements = case['requirements']
#requirements.max_energy = -1

In [9]:
from combined_problem import ExtendedCombinedProblem
problem = ExtendedCombinedProblem(instances, sys_param, requirements=requirements, f_mask=np.array([0, 2, 3]))

sampling, crossover, mutation = design_vector_default_scm(problem.x_length, problem.x_indices)

algo_settings = {'pop_size': 200, 'n_offsprings': 50, 'n_max_gen': 5000}

settings = {}

# U-NSGA-III
from pymoo.factory import get_reference_directions, get_visualization, get_termination

ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=15)
print("# dirs: %d" % len(ref_dirs))

settings['UNSGA-III'] = {
    'label': 'unsga3',
    'algorithm': UNSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    # 'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
    #                                                   n_last=30,
    #                                                   nth_gen=5,
    #                                                   n_max_gen=algo_settings['n_max_gen'],
    #                                                   n_max_evals=None)
    'termination': get_termination("time", "01:00:00")
}

# NSGA-III
ref_dirs = get_reference_directions("das-dennis", 3, n_partitions=23)
print("# dirs: %d" % len(ref_dirs))
settings['NSGA-III'] = {
    'label': 'nsga3',
    'algorithm': NSGA3(
        pop_size=int(np.ceil(len(ref_dirs) / 100) * 100),
        n_offsprings=int(np.ceil(len(ref_dirs) / 100) * 100 / 4),
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        ref_dirs=ref_dirs,
        eliminate_duplicates=True,
    ),
    'termination': get_termination("time", "01:00:00")
}

from pymoo.algorithms.rnsga2 import RNSGA2

ref_points = np.array([[case['T_bitorbit_target'], case['L_sorbit_max'], case['E_Jorbit_target'], 0.4*case['T_orbit_s']]])
ref_points = ref_points * (case['T_sim_s'] / case['T_orbit_s'])
ref_points = ref_points *  np.array([-1, 1, 1, 1])
ref_points = ref_points[:, problem.f_mask]

settings['RNSGA-II-0-001'] = {
    'label': 'rnsga2_e0_001',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.001,
        normalization='front',
        extreme_points_as_reference_points=False,
        weights=np.array([0.5, 0.5, 0])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

settings['RNSGA-II-0-01'] = {
    'label': 'rnsga2_e0_01',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.01,
        normalization='front',
        extreme_points_as_reference_points=False,
        #weights=np.array([0.5, 0.5, 0.5])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

settings['RNSGA-II-0-0001'] = {
    'label': 'rnsga2_e0_0001',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=0.0001,
        normalization='front',
        extreme_points_as_reference_points=False,
        weights=np.array([0.5, 0.5, 0])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

settings['RNSGA-II-1e-9'] = {
    'label': 'rnsga2_e1e-9',
    'algorithm': RNSGA2(
        pop_size=algo_settings['pop_size'],
        n_offsprings=algo_settings['n_offsprings'],
        sampling=sampling,
        crossover=crossover,
        mutation=mutation,
        eliminate_duplicates=True,
        ref_points=ref_points,
        epsilon=1e-9,
        normalization='front',
        extreme_points_as_reference_points=False,
        weights=np.array([0.5, 0.5, 0])
    ),
    'termination': MultiObjectiveSpaceToleranceTermination(tol=0.00001,
                                                      n_last=30,
                                                      nth_gen=5,
                                                      n_max_gen=5000,
                                                      n_max_evals=None)
}

setting = settings['UNSGA-III']
setting = settings['NSGA-III']
#setting = settings['RNSGA-II-0-01']
#setting = settings['RNSGA-II-0-001']
#setting = settings['RNSGA-II-0-0001']
#setting = settings['RNSGA-II-1e-9']

# dirs: 136
# dirs: 300


In [15]:
termination = setting['termination']
algorithm = setting['algorithm']

n_gen_inc = 1000

from os import path

file_name = 'raw/eo_Telesat_%s_%d_%d.pkl' % (setting['label'], algorithm.pop_size, algorithm.n_offsprings)

if path.exists(file_name):
    problem, setting, res = pickle.load(open(file_name, 'rb'))
    algorithm = res.algorithm
    algorithm.has_terminated = False
    n_gen = res.algorithm.n_gen
else:
    n_gen = 0

In [16]:
n_gen = n_gen + n_gen_inc
print(file_name)
print(n_gen)
termination = ('n_gen', n_gen)

nsga3
1000


In [17]:
res = minimize(problem,
               algorithm,
               termination,
               #termination=termination,
               seed=1,
               #save_history=True,
               verbose=True,
               copy_algorithm=False,
               )

print('Processes:', res.exec_time)
print("Best solution found: %s" % res.X)

pickle.dump((problem, setting, res), open(file_name, 'wb'))

n_gen |  n_eval |   cv (min)   |   cv (avg)   |  n_nds  |     eps      |  indicator  
    1 |     600 |  0.00000E+00 |  0.737223430 |      11 |            - |            -
    2 |     675 |  0.00000E+00 |  0.615800316 |      11 |  0.019647357 |            f
    3 |     750 |  0.00000E+00 |  0.507188724 |      15 |  0.121228094 |        ideal
    4 |     825 |  0.00000E+00 |  0.392421970 |      15 |  0.028518074 |        ideal
    5 |     900 |  0.00000E+00 |  0.283449425 |      17 |  0.165372671 |        ideal
    6 |     975 |  0.00000E+00 |  0.185266685 |      18 |  0.187894073 |        nadir
    7 |    1050 |  0.00000E+00 |  0.112647900 |      20 |  0.020781597 |            f
    8 |    1125 |  0.00000E+00 |  0.058656225 |      22 |  0.249082264 |        nadir
    9 |    1200 |  0.00000E+00 |  0.027280805 |      21 |  0.018885808 |            f
   10 |    1275 |  0.00000E+00 |  0.007363744 |      24 |  0.033112795 |            f
   11 |    1350 |  0.00000E+00 |  0.001185953 |      2

In [142]:
from beepy import beep
for i in range(3):
    beep(sound=5)
